# Objective

The objective of this repository is to build a machine learning model that is capable of making predictions on the movement of asset prices. Once a model is made that is of a high enough standard, it will be deployed into production to make predictions on an ongoing basis. The purpose of this documentation is to compliment the code and provide a different way of understanding the nature of the project. It will also provide me with a summary of what I have tried and am yet to try.

My eventual aim is to do the above for several assets in order to increase the opportunity for the model(s) to provide value and use them in tandem to exploit the different movements of different assets.

# Tools

The main tool I will be using is Databricks. It has a few features which I think will be useful for me including:
- Parallel Computing: I will be learning to use pyspark (Python API on Spark computing) in order to distribute my operations between nodes, allowing for more efficient querying and transforming on the data, which could be fairly large considering how far back I would like to go
- AutoML: I would take advantage of the AutoML offering in order to build baseline models for each dataset I use, giving me a fast way of knowing what the predictive power of my data might be
- MLflow: I will be leaning heavily on MLflow in order to track experiments and different models I build so that I know what direction to go in and know whether a model I built is the best yet; it will also play a large role when it comes to deployment

I am also hoping to utilise Numba, a Python library (built into Databricks ML Runtime) that is capable of making my code more efficient when it comes to executing. It does this by making full use of the NVIDIA GPU attached to the cluster to distribute work efficiently between threads. 

Another tool I am interested in is testing for my code. I want to implement testing into my notebook so that I can quickly see where my code isn't outputting the result I expected. This is important as mistakes in price predictions can be costly if acted upon and so robustness and correctness of my code is of high importance.

I will consistently be using Git as I complete this project. This will give me a chance to version my notebooks and return to previous should I make a mistake, it will allow others to contribute to the repository if it gets that far and it allows me to show my work publicly through my GitHub account.

# Process

## Data

### Using Spark

To start, I am creating a Spark Session. This will allow me to utilise the computing architecture of Apache Spark, for example by creating dataframes that can be parallelized for operations, transformations and querying. I am also creating  a database 'aaronxwalker' in order to store my curated data and different versions of data.

I am using the Python package yfinance to import price history for Apple. This is because it has a sufficient amount of liquidity to trade and data is readily available. To begin with I am getting data from the start of 2015 till now; this will hopefully provide me with enough meaningful data to build some complex models but also being slightly faster than working on more data and I do have the option to go back for more should I need it.

### Target

My target column is the percentage change of the stock price from the close of day t-1 to the close of day t. I have chosen Close because it gives me a chance to bet on the outcome whilst the market is open, whereas betting on the Open means I can't take advantage of a difference in price overnight as the market is shut. The reason I am using percentage change is that it is indifferent to the price of the stock, which vary among stocks but has no meaning. It also is likely to have a recognisable distribution which might come in useful to certain parametric models later on.

I will also look at the performance of predicting on log returns, which supposedly have even more of a normal distribution and might strengthen the results of some models. It also has an additive property meaning I can simply add the log returns of several days to get the cumulative returns for those days. However, I have yet to test this target.

### Features

To begin with, I am using the features that are provided by yfinance. This includes basic time-series information on the stock price, including open, high, low, volume, dividends and stock splits.

#### Lagging Features

It's important not to use features that we won't have access to in time for inference. For example, we don't want to use the volume for day t to make predictions on the target for day t as the volume won't be accessable till the end of the day, by which time it will be too late to make a prediction. To this end, I have decided to lag each of the above features by a day so that I can use the previous days information to make inferences for the next day.

# Architecture

## Pandas

To do all my preliminary work on the data, I used a Pandas dataframe to apply: creating target variable, shifting features, reset index (spark dataframes don't have an index) and renaming columns to be lower case and have no spaces.

### Train-test Split

I used Pandas to split my data up. As I'm working with time-series data here, I can't do a random split as the test data would be able to learn from future prices to make predictions, known as data leakage. Therefore, I am splitting up my data by time, with the first 70% used to train my model, the next 15% used to validate the models results and the final 15% used to evaluate the performance of my model on unseen data. I have gone for a 70-15-15 split as I only have a few thousand datapoints and so I want to give enough data to val and test to get robust results. For each row of my validation and test, I plan to use all of the data available up until the day previous to each data point.

## Spark

I then converted my train, val and test dataframes to Spark dataframes which can utilise parallelization. I saved these dataframes as 3 seperate tables in my database under the names `aapl_returns_{train/val/test}` and updated each of their comments box with a little description about the data in the table such as the target, the features and the intended use for the data. As I grow each particular dataset I can overwrite the table, creating a new version that might be for testing or the best data I have yet. This also allows me to rollback a table if I find the newest version isn't beneficial to my model.

# AutoML

In order to build a baseline model to get some sort of idea of how much predictive power our features have, I'll be using AutoML to quickly understand my starting point. AutoML on Databricks is going to provide me with a fast way to test different models and see which performs best. It also allows me to use this model to make predictions, see a notebook of the model built and EDA that it done and look at various metrics to see how predictive the model is.

## Configuring

In the configuration for AutoML, I'm selecting Forecasting as the ML problem type as I'm working with time-series data and wanting to predict the future, I'm using my train dataset as this is what AutoML is using to train the models, I select the column that is my date column as this is the only feature it uses in Forecasting and I'm predicting 100 steps into the future. I'm hoping that I will be able to see how the robustness of the model varies predicting several steps forwards which should give me an idea on how often I will need to retrain the model with updated data.

For evaluation metric, I have selected SMAPE as that's considered a good metric for time-series analysis. Regardless of which I select, all pre-determined metric choices are evaluated when modelling is done and so I will have access to all of their results. There are only 2 forecasting models available (ARIMA and Prophet) and so I will be selecting both for AutoML to try. I can also enter Country Holidays so I have selected US as I am working with a US company. I have selected 20 minutes timeout as it is the end of the day and I don't want to be waiting too long, its only an initial insight.

### Experiments

For this AutoML experiment, I am creating a new experiment named `aapl_returns_forecasting_automl`. In this experiment I will record runs that are done by AutoML and update their versions as each run improves with data or modelling. I also add a description for my experiment to explain what runs in the experiment do.

## Results

So AutoML was only able to train Prophet models as I hadn't preprocessed the data to have the desired frequency but this isn't an issue as I can still get a rough benchmark from Prophet. It trained 4 different Prophet models in about 8 minutes and has evaluated them on some hidden validation set using SMAPE; I expect I can see the validation set used by going into the provided data exploration notebook or notebook for best model. All of these runs are saved in my experiment I created as well as the notebooks mentioned, the prediction results for the best model (100 steps ahead) and the metrics for all of the models.

### Runs

Going into the run of the best model (based on val_smape), I have access to the notebook to see what was done to the data, I can get the results for the future forecasting and make predictions on new dates. I'm importing the predictions for the next 100 steps/days (confusingly this is now testing on my validation set; autoML made its own validation set from the train set) and visualising them using the UI visualisation tool for Spark DataFrames. We see a small upwards trend in returns over the 100 days which could be a meaningful feature of the data to return to later on, and a weekly seasonality to the predictions. Upon further inspection, autoML doesn't provide an option and will make weekend predictions which are where the peaks of the weekly seasonality occur, so these can be discounted. **I need to check if these skew the results upon being removed or for some reason are abnormal returns that autoML has decided to create.**

**However when we plot our predicted returns vs our actual returns, on the whole we can see that the variance in daily returns is almost non-existant and the best the Prophet model can do is essentially predict the same thing every day, the mean of the returns over the given time period.**

## Ideas to improve model performance

When I'm struggling to improve the performance of my model, here are some ideas I plan to try:
- increase the number of useful features in my model (gather more external data)
- try to split up the data into different periods so that all the data used comes from the same distribution
- make a classification model to predict +/- returns for the day
- change the time period I am predicting over e.g. predict returns over the next month instead of next day
- explore different models
- read a book on time-series
- try a different asset

## In Production